<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Tarea Tutorial - Semana 4 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana4.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>REINFORCE</h1></center>

Este tutorial pretende introducir uno de los primeros algoritmos de gradiente de política: REINFORCE. Nuevamente, para ilustrar el funcionamiento y desempeño del algoritmo, se hará uso de un problema basado en un juego de Atari, donde el agente aprenderá directamente de las imágenes del juego. Al mismo tiempo, se profundizará en los beneficios que tiene la utilización de <i>baselines</i> durante el entrenamiento. Para ello, en este notebook se utiliza la librería Gymnasium, que incluye el juego de ________________, y una implementación de REINFORCE realizada para PyTorch.


# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#scrollTo=Objetivos_de_Aprendizaje)  
2. [Marco Teórico](#scrollTo=Marco_Te_rico)  
3. [Instalación de Librerías](#scrollTo=Instalaci_n_de_Librer_as)  
4. [Familiarización con el Entorno de Gym](#scrollTo=Familiarizaci_n_con_el_Entorno_de_Gym)  
5. [REINFORCE](#scrollTo=REINFORCE)
6. [REINFORCE con <i>baseline</i>](#scrollTo=REINFORCE_con_baseline)
7. [Reflexiones Finales](#scrollTo=Reflexiones_Finales)  
8. [Referencias](#scrollTo=Referencias)

# Objetivos de Aprendizaje  

Este tutorial tiene como objetivo:
  

*   Intorducir una familia distinta de algoritmos de aprendizaje por refuerzo: algoritmos de gradiente de política.
*   Comprender las bases teóricas y práticas del funcionamiento del algoritmo de REIFORCE.
*   Exponer las ventajas que tiene la inclusión de <i>baselines</i> dentro del entrenamiento de algoritmos de aprendizaje por refuerzo.

# Marco Teórico  

Hasta ahora, los algoritmos de aprendizaje por refuerzo que se han explorado se basan en aprender y estimar una función de valor para cada acción, por lo que reciben el nombre de métodos de valor-acción (<i>action-value methods</i>). Es decir, las políticas aprendidas en estos métodos dependen directamente de los estimativos de la función de valor-acción ($Q(s,a)$). Sin embargo, existe otra familia de algoritmos que siguen un método de gradiente de política (<i>policy gradient methods</i>), que directamente aprenden una política parametrizada con la capacidad de seleccionar acciones sin la necesidad de estimar una función de valor. Una función de valor puede ser todavía utilizada para aprender el parámetro de la política, pero no se requiere para seleccionar la acción. [1]

Por convención de notación, se utiliza $\theta$ para referirse a un vector de parámetros de la política. Entonces se puede escribir que la política, corresponde a la probabilidad de elegir una acción dado un estado y el vector de parámetros: $\pi(a|s,\theta) = Pr(A_t=a|S_t=s, \theta_t=\theta)$. La forma en que un algoritmo aprende y actualiza el vector de parámetros $\theta$ se basa en el gradiente de alguna métrica de desempeño escalar $J(\theta)$. Este método busca maximizar el desempeño, así que se tiene un ascenso de gradiente en $J$, como se muestra en la Ecuación (1). Se utiliza la notación $\widehat{\nabla J(\theta_t)}$ para denotar que el gradiente en realidad se calcula utilizando un estimativo estocástico. [1]

<center> $\theta_{t+1}=\theta + \alpha \widehat{\nabla J(\theta_t)}$ &emsp;&emsp;&emsp;$(1)$ </center>

Otro elemento importante a tener en cuenta es el teorema de gradiente de política (policy gradient theorem), mostrado en la Ecuación (2). En este teorema, se establece que existe una proporcionalidad entre el gradiente de la función $J(\theta)$ y una expresión analítica dependiente de los parámetros de la política ($\theta$) y que se puede muestrear directamente a partir de la interacción con el ambiente a medida que se mejora la política.
Esto en últimas permite encontrar la máxima dirección de crecimiento de $J(\theta)$ y ajustar los parámetros utilizando una constante de proporcionalidad $\alpha$ como se escribió en la Ecuación (1).

<center> $\nabla J(\theta) \propto \sum_s \mu(s) \sum_a q_\pi(s, a) \nabla \pi(a|s, \theta)$ &emsp;&emsp;&emsp;$(2)$ </center>

El algoritmo de REINFORCE, un algoritmo de gradiente de política, utiliza la regla de actualización de la Ecuación (3), estimando el gradiente estocástico utilizando el retorno $G_t$:

<center> $\theta_{t+1} \doteq \theta_t + \alpha G_t \frac{\nabla \pi(A_t | S_t, \theta_t)}{\pi(A_t | S_t, \theta_t)}$ &emsp;&emsp;&emsp;$(3)$ </center>

En esta actualización, cada incremento es proporcional al producto del retorno $G_t$ y un vector: el gradiente de la probabilidad de escoger la acción tomada dividida por la probabilidad de escoger dicha acción. Este vector corresponde a la dirección en el espacio de parámetros en la cual crece mayormente la probabilidad de escoger la acción $A_t$ en futuras visitas al estado $S_t$. La actualización es proporcional al retorno $G_t$, favoreciendo las acciones que generan mayor recompensa a largo plazo, y es inversamente proporcional a la probabilidad de escoger la acción, no favoreciendo así incorrectamente a acciones que se eligen frecuentemente. Debido a que REINFORCE utiliza el retorno completo desde un tiempo $t$, que incluye todas las recompensas vistas hasta el final del episodio, se dice que entonces REINFORCE es también un método de Monte Carlo. [1]

Finalmente, se consigue una variación de la regla de actualización de REINFORCE si se incluye un <i>baseline</i>, como muestra la Ecuación (4). Incluir un baseline no cambia el valor esperado de la actualización, pero sí puede tener un efecto significativo en la varianza del aprendizaje, reduciéndola significativamente y haciendo el entrenamiento más estable. Para el caso de un MDP, dicho baseline debería variar con el estado, por lo cual una selección típica es utilizar una función de valor de estado $\hat{v}(s,\text{w})$, donde $\text{w}$ es un vector de pesos aprendido por otro algoritmo de valor-acción. [1]

<center> $\theta_{t+1} \doteq \theta_t + \alpha (G_t-b(S_t)) \frac{\nabla \pi(A_t | S_t, \theta_t)}{\pi(A_t | S_t, \theta_t)}$ &emsp;&emsp;&emsp;$(4)$ </center>



# Instalación de Librerías  

En este tutorial se va a utilizar un juego de Atari 2600: <i>______</i>. Este videojuego ya se encuentra incluido en los ambientes de Atari de la librería Gymnasium. También se requiere instalar correctamente PyTorch para el uso de redes neuronales.

Antes de comenzar, se sugiere elegir un entorno de simulación acelerado por GPU. En el caso de Colab gratuito, debería elegir el entorno de T4. Para ello diríjase a:

`Entorno de Ejecución > Cambiar Tipo de Entorno de Ejecución > GPU T4`


![DobleDQNdF](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/t4.png)


Después, ejecute el siguiente bloque de código para instalar todas las librerías y herramientas necesarias.


In [1]:
#Descarga librerías no incluidas en Colab usando pip
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata #Para compatibilidad de PyTorch
!pip3 install torch torchaudio torchvision torchtext torchdata #Instala PyTorch
!pip install ale-py #ALE se utiliza para el ambiente de Atari
!pip install "gymnasium[atari,accept-rom-license]" stable-baselines3 autorom renderlab -q #Gymnasium, envs de Atari y ROM
!AutoROM --accept-license
!pip install renderlab #usado para renderizar gym
!pip install stable_baselines3 #Stable Baselines3 -> Framework de Reinforcement Learning

#Importa estas librerías
import torch #Importa herramientas de PyTorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import gymnasium #importa la libreria de gymnasium con las simulaciones
import renderlab #importa renderlab para los videos
import stable_baselines3 #importa Stable Baselines3
from stable_baselines3.common.env_util import make_atari_env #importa make_atari_env para escala de grises
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack #importa VecFrameStack para apilar frames y acelerar así el entrenamiento
import ale_py #importa ale para los ambientes de Atari
from gymnasium.wrappers import TimeLimit #importa timelimit para acortar los episodios
from collections import deque #importa para ajustar los videos con VecFrameStack
import cv2 #importa para ajustar los videos con VecFrameStack

#!Importante:
gymnasium.register_envs(ale_py) #Hay que registrar los entornos de ALE manualmente!!!

#Importa otras librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys
import argparse
from itertools import count

#Limpia los registros generados
from IPython.display import clear_output
clear_output()
print("Todas las librerías han sido instaladas correctamente.")

Todas las librerías han sido instaladas correctamente.


# Familiarización con el Entorno de Gym

## Ejemplo



In [6]:
#Parámetros del ambiente
mode=0
difficulty=0

env_render = gymnasium.make("ALE/Breakout-v5", render_mode="rgb_array", mode=mode, difficulty=difficulty) #Se crea el ambiente.
env_render = TimeLimit(env_render, max_episode_steps=500)
env_render = renderlab.RenderFrame(env_render, "./output") #Se crea una copia que se pueda renderizar con renderlab


terminated = False #Inicializa una condición para el loop
truncated = False #Inicializa una condición para el loop
total_reward=0 #Inicializa contador del retorno

obs , info = env_render.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 2 dimensiones)
obs, reward, terminated, truncated, info = env_render.step(1) #Se iuni
while not (terminated or truncated): #Simula hasta que termine la partida
  action = random.choice([0, 1, 2, 3])  # Elige una acción aleatoria
  obs, reward, terminated, truncated, info = env_render.step(action)
  total_reward += reward

print("Recompensa obtenida en el episodio:",total_reward) #Se imprime la recompensa obtenida
print("\n\n")

env_render.play() #Con esta función se obtiene el video de la simulación

Recompensa obtenida en el episodio: 1.0



Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



t:  61%|██████    | 111/183 [00:00<00:00, 1105.35it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./output/1746902807.4008334.mp4, 100800 bytes wanted but 0 bytes read,at frame 182/183, at time 6.07/6.07 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


In [ ]:
# Ejecute una partida con dificultad 1 y modo 7

# =====================================================
# COMPLETAR ===========================================
#

# =====================================================

#REINFORCE



In [7]:

class Policy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(input_dim, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, output_dim)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

class REINFORCE:
  def __init__(self,env_name, use_baseline=True, max_steps_per_episode=10000):
    self.gamma=0.99
    self.seed=543
    self.render=False
    self.log_interval=10
    self.env_name=env_name
    self.max_steps_per_episode=max_steps_per_episode
    self.use_baseline=use_baseline


    self.env = gymnasium.make(self.env_name)
    obs, _ = self.env.reset(seed=self.seed)
    torch.manual_seed(self.seed)

    input_dim = obs.shape[0]
    if hasattr(self.env.action_space, 'n'):
      output_dim = self.env.action_space.n
    else:
      raise ValueError("El espacio de acciones debe ser discreto.")

    self.policy = Policy(input_dim, output_dim)
    self.optimizer = optim.Adam(self.policy.parameters(), lr=1e-2)
    self.eps = np.finfo(np.float32).eps.item()

  def select_action(self, state):
      state = torch.from_numpy(state).float().unsqueeze(0)
      probs = self.policy(state)
      m = Categorical(probs)
      action = m.sample()
      self.policy.saved_log_probs.append(m.log_prob(action))
      return action.item()
  """
  def finish_episode(self):
      R = 0
      policy_loss = []
      returns = deque()
      for r in self.policy.rewards[::-1]:
          R = r + self.gamma * R
          returns.appendleft(R)
      returns = torch.tensor(returns)
      returns = (returns - returns.mean()) / (returns.std() + self.eps)
      for log_prob, R in zip(self.policy.saved_log_probs, returns):
          policy_loss.append(-log_prob * R)
      self.optimizer.zero_grad()
      policy_loss = torch.cat(policy_loss).sum()
      policy_loss.backward()
      self.optimizer.step()
      del self.policy.rewards[:]
      del self.policy.saved_log_probs[:]
  """
  def finish_episode(self):
    #Version con Baselines
    R = 0
    policy_loss = []
    returns = deque()

    # Calculamos el baseline como el promedio de las recompensas (ESTO HAY QUE CAMBIARLO A ALGO MÁS ÚTIL!!!!)
    if(self.use_baseline):
      baseline = torch.mean(torch.tensor(self.policy.rewards, dtype=torch.float32))
    else:
      baseline=0 #Si self.use_baselines es False, coloca baseline en 0 que es efectivavmente no usar baselines

    for r in self.policy.rewards[::-1]:
        R = r + self.gamma * R
        returns.appendleft(R)
    returns = torch.tensor(returns)

    # Normalizamos las recompensas con respecto al baseline
    returns = returns - baseline  # Descontamos el baseline de las recompensas

    returns = (returns - returns.mean()) / (returns.std() + self.eps)
    for log_prob, R in zip(self.policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    self.optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    self.optimizer.step()
    del self.policy.rewards[:]
    del self.policy.saved_log_probs[:]

  def train(self):
    running_reward = 10
    for i_episode in count(1):
        state, _ = self.env.reset()
        ep_reward = 0
        for t in range(1, self.max_steps_per_episode):  # Don't infinite loop while learning
            action = self.select_action(state)
            state, reward, done, _, _ = self.env.step(action)
            self.policy.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        self.finish_episode()
        if i_episode % self.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if hasattr(self.env.spec, 'reward_threshold') and running_reward > self.env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

  def video(self):
    env_prueba_1 = gymnasium.make(self.env_name, render_mode="rgb_array") #Esta línea de código crea el ambiente.
    env_prueba_1 = renderlab.RenderFrame(env_prueba_1, "./output") #Esta línea se utiliza para crear una copia que se pueda renderizar con renderlab

    obs , info = env_prueba_1.reset() #Se reinicia el estado para comenzar. En obs se almacena el estado observado (continuo, 4 dimensiones)
    terminated = False #Inicializa una condición para el loop
    truncated = False #Inicializa una condición para el loop
    total_reward=0 #Inicializa contador del retorno

    while not (terminated or truncated): #Simula hasta que el poste caiga o hasta alcanzar 500 episodios (configuración de CartPole-v1)
      action=self.select_action(obs)
      obs, reward, terminated, truncated , info = env_prueba_1.step(action) #Con la función step el ambiente da un paso. Se obtiene el estado, recompensa y banderas de información
      total_reward+=reward #Llevamos una cuenta de la recompensa total


    print("\n\n\n\n")
    print("Recompensa obtenida en el episodio:",total_reward) #Después de terminar el episodio, imprimios la recompensa acumulada total obtenida
    print("\n\n")

    env_prueba_1.play() #Con esta función se obtiene el video de la simulación


print("Sin Baseline")
agente_cartpole=REINFORCE('CartPole-v1', use_baseline=False)
agente_cartpole.train()
agente_cartpole.video()

print("Con Baseline")
agente_cartpole2=REINFORCE('CartPole-v1')
agente_cartpole2.train()
agente_cartpole2.video()


Sin Baseline
Episode 10	Last reward: 40.00	Average reward: 16.52
Episode 20	Last reward: 13.00	Average reward: 18.72
Episode 30	Last reward: 63.00	Average reward: 22.55
Episode 40	Last reward: 93.00	Average reward: 36.51
Episode 50	Last reward: 109.00	Average reward: 51.41
Episode 60	Last reward: 36.00	Average reward: 60.96
Episode 70	Last reward: 183.00	Average reward: 84.92
Episode 80	Last reward: 64.00	Average reward: 88.42
Episode 90	Last reward: 114.00	Average reward: 100.46
Episode 100	Last reward: 205.00	Average reward: 128.58
Episode 110	Last reward: 64.00	Average reward: 134.44
Episode 120	Last reward: 122.00	Average reward: 123.34
Episode 130	Last reward: 98.00	Average reward: 98.67
Episode 140	Last reward: 53.00	Average reward: 78.16
Episode 150	Last reward: 94.00	Average reward: 79.14
Episode 160	Last reward: 330.00	Average reward: 130.43
Episode 170	Last reward: 157.00	Average reward: 197.36
Solved! Running reward is now 508.69395670690153 and the last episode runs to 2055

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


Con Baseline
Episode 10	Last reward: 40.00	Average reward: 16.52
Episode 20	Last reward: 13.00	Average reward: 18.72
Episode 30	Last reward: 63.00	Average reward: 22.55
Episode 40	Last reward: 93.00	Average reward: 36.51
Episode 50	Last reward: 109.00	Average reward: 51.41
Episode 60	Last reward: 36.00	Average reward: 60.96
Episode 70	Last reward: 183.00	Average reward: 84.92
Episode 80	Last reward: 64.00	Average reward: 88.42
Episode 90	Last reward: 43.00	Average reward: 99.33
Episode 100	Last reward: 126.00	Average reward: 102.82
Episode 110	Last reward: 148.00	Average reward: 115.30
Episode 120	Last reward: 153.00	Average reward: 115.65
Episode 130	Last reward: 222.00	Average reward: 137.11
Episode 140	Last reward: 207.00	Average reward: 141.53
Episode 150	Last reward: 344.00	Average reward: 186.50
Episode 160	Last reward: 805.00	Average reward: 344.39
Solved! Running reward is now 476.2341497208486 and the last episode runs to 1301 time steps!





Recompensa obtenida en el episodi

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


#REINFORCE con baseline

# Reflexiones Finales






# Referencias

[1] Sutton, R. S. and Barto, A. G. (2018). Reinforcement Learning: An Introduction. The MIT Press, second edition.

[2] Gym Documentation, Freeway. `https://gymnasium.farama.org/v0.28.1/environments/atari/freeway/`

[3] PyTorch REINFORCE example. `https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py`